In [1]:
!pip install --quiet --upgrade langchain langchain-community langchain-chroma langchain-text-splitters langchain-google-vertexai

In [2]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "default"

In [3]:
print(os.environ["LANGCHAIN_PROJECT"])

default


In [4]:
PROJECT_ID = "test-rag-sep2024"
REGION = "us-central1"

from google.colab import auth
auth.authenticate_user()

In [5]:
import vertexai
vertexai.init(project = PROJECT_ID , location = REGION)

In [6]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-1.5-flash")

In [7]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embeddings = VertexAIEmbeddings(model_name="text-embedding-004")


In [10]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [11]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

In [12]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [13]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
rag_chain.invoke("Describe how LLM agents work using chain of thought?")

'LLM agents leverage chain of thought (CoT) to break down complex tasks into smaller, manageable steps. This allows the model to "think step by step" and use more computational resources during task execution. By decomposing large tasks, CoT aids in interpreting the model\'s reasoning process. \n'